<a href="https://colab.research.google.com/github/agustinsilva447/CDL/blob/master/mnist_hyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip3 install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

/home/alan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/alan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/alan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/alan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

In [7]:
#función para graficar el accuracy en función de la cantidad de epochs
def plot_accuracy_and_loss(history):
	plt.plot(history.history['accuracy'], label='accuracy')
	plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
	plt.xlabel('Epoch')
	plt.ylabel('Accuracy')
	plt.ylim([0.5, 1])
	plt.legend(loc='lower right')

In [8]:
#cargando el dataset MNIST desde keras
(x_train, y_train), (x_test, y_test) = mnist.load_data()
#normalizando los valores de los pixels para que estén entre 0 y 1
x_train, x_test = x_train / 255.0, x_test / 255.0

In [9]:
# Todas estas son la posibles variables que se pueden autoconfigurar
# sin embargo, solo vamos a trabajar con 3 para que las simulaciones
# no lleven tanto tiempo. Vamos a trabajar con una red neuronal 
# con 2 hidden layers y optimizada utilizando stochastic gradient descent. 
# Vamos a tunear 3 parámetros: el learning rate, la cantidad de neuronas 
# por hidden layer y la cantidad de epochs.
lr_ = [0.001, 0.01, 0.1]		###tres casos usados
hidden_units_ = [16, 32, 64]	###tres casos usados
epochs_ = [5, 10]				##dos casos usados
batch_size_ = 64				#un solo caso usado
momentum_ = 0.9					#un solo caso usado
decay_ = 1e-6					#un solo caso usado
hidden_layers_ = 2				#un solo caso usado
nesterov_ = True				#un solo caso usado

In [10]:
#inicialización de algunas variables
test_loss = [None] * (len(hidden_units_) * len(epochs_) * len(lr_))
test_acc =  [None] * (len(hidden_units_) * len(epochs_) * len(lr_))
caso = 0
max_acc = 0

In [11]:
# Barremos con los distintos valores de neuronas (3), epochs (2) 
# y learning rate (3). Para cada caso calculamos la accuracy y 
# nos quedamos con el caso con el valor más alto
for i in range(len(hidden_units_)):
  for j in range(len(epochs_)):
    for k in range(len(lr_)):
			#modelo con 2 hidden layers
      model = Sequential([
      layers.Flatten(input_shape=(28, 28)),
			layers.Dense(hidden_units_[i], activation='relu'),
			layers.Dense(hidden_units_[i], activation='relu'),
			layers.Dense(10, activation='softmax')
      ])
      
			#seteado de los parámetros del stochastic gradient descent
      sgd = optimizers.SGD(lr=lr_[k], 
                  decay=decay_, 
                  momentum = momentum_, 
                  nesterov=nesterov_)
      model.compile(optimizer=sgd, 
            loss='sparse_categorical_crossentropy', 
            metrics=['accuracy'])

      # entrenamiento de la red neuronal
      history = model.fit(x_train, 
                      y_train, 
                      epochs=epochs_[j], 
                      batch_size=batch_size_, 
                      validation_data=(x_test, y_test))
                      
      # evaluación de la red para la entrada del test set 
      # comparado con la verdaderas salidas
      test_loss[caso], test_acc[caso] = model.evaluate(x_test,  
                                                    y_test, verbose=2)  
      
      # nos quedamos con el valor de accuracy más alto
      if test_acc[caso]>max_acc:
        max_acc = test_acc[caso]
        neu_opt = hidden_units_[i]
        epo_opt = epochs_[j]
        lra_opt = lr_[k]
        hist_opt = history
      caso += 1

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 2s 27us/sample - loss: 1.3708 - acc: 0.5806 - val_loss: 0.6592 - val_acc: 0.8236
Epoch 2/5
60000/60000 [==============================] - 2s 27us/sample - loss: 0.5156 - acc: 0.8552 - val_loss: 0.4048 - val_acc: 0.8817
Epoch 3/5
60000/60000 [==============================] - 2s 29us/sample - loss: 0.3846 - acc: 0.8907 - val_loss: 0.3420 - val_acc: 0.8992
Epoch 4/5
60000/60000 [==============================] - 2s 30us/sample - loss: 0.3398 - acc: 0.9022 - val_loss: 0.3132 - val_acc: 0.9091
Epoch 5/5
60000/60000 [==============================] - 2s 39us/sample - loss: 0.3155 - acc: 0.9099 - val_loss: 0.2973 - val_acc: 0.9132
10000/10000 - 0s - loss: 0.2973 - acc: 0.9132
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [=======================

60000/60000 [==============================] - 2s 29us/sample - loss: 0.1976 - acc: 0.9408 - val_loss: 0.1796 - val_acc: 0.9438
Epoch 3/5
60000/60000 [==============================] - 2s 33us/sample - loss: 0.1552 - acc: 0.9542 - val_loss: 0.1532 - val_acc: 0.9538
Epoch 4/5
60000/60000 [==============================] - 2s 26us/sample - loss: 0.1287 - acc: 0.9613 - val_loss: 0.1434 - val_acc: 0.9561
Epoch 5/5
60000/60000 [==============================] - 2s 31us/sample - loss: 0.1128 - acc: 0.9657 - val_loss: 0.1202 - val_acc: 0.9621
10000/10000 - 0s - loss: 0.1202 - acc: 0.9621
Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 2s 29us/sample - loss: 0.3010 - acc: 0.9109 - val_loss: 0.1979 - val_acc: 0.9416
Epoch 2/5
60000/60000 [==============================] - 2s 31us/sample - loss: 0.1941 - acc: 0.9440 - val_loss: 0.1797 - val_acc: 0.9456
Epoch 3/5
60000/60000 [==============================] - 2s 31us/sample - loss: 0.1730

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 41us/sample - loss: 1.0087 - acc: 0.7302 - val_loss: 0.4277 - val_acc: 0.8818
Epoch 2/10
60000/60000 [==============================] - 2s 41us/sample - loss: 0.3814 - acc: 0.8928 - val_loss: 0.3202 - val_acc: 0.9084
Epoch 3/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.3158 - acc: 0.9093 - val_loss: 0.2833 - val_acc: 0.9187
Epoch 4/10
60000/60000 [==============================] - 2s 35us/sample - loss: 0.2825 - acc: 0.9193 - val_loss: 0.2563 - val_acc: 0.9274
Epoch 5/10
60000/60000 [==============================] - 2s 39us/sample - loss: 0.2595 - acc: 0.9259 - val_loss: 0.2379 - val_acc: 0.9317
Epoch 6/10
60000/60000 [==============================] - 2s 34us/sample - loss: 0.2408 - acc: 0.9311 - val_loss: 0.2247 - val_acc: 0.9357
Epoch 7/10
60000/60000 [==============================] - 2s 34us/sample - loss: 0.2251 - acc: 0.9353 - val_loss: 0.

In [12]:
#mostrar resumen de todos los resultados y finalmente el caso óptimo
caso = 0
for i in range(len(hidden_units_)):
	for j in range(len(epochs_)):
		for k in range(len(lr_)):
			print("Cantidad de neuronas:",hidden_units_[i],". Epochs:",epochs_[j],". Learning rate:",lr_[k],". Accuracy:", test_acc[caso], ". Loss:", test_loss[caso])
			caso += 1
print("---------------------------------------------")
print("El Accuracy máximo es:",max_acc,". Para el caso con",neu_opt,"cantidad de neuras,",epo_opt,"epocs y un learning rate de",lra_opt)

Cantidad de neuronas: 16 . Epochs: 5 . Learning rate: 0.001 . Accuracy: 0.9132 . Loss: 0.29725642626583576
Cantidad de neuronas: 16 . Epochs: 5 . Learning rate: 0.01 . Accuracy: 0.9493 . Loss: 0.1781372046675533
Cantidad de neuronas: 16 . Epochs: 5 . Learning rate: 0.1 . Accuracy: 0.9393 . Loss: 0.23093445731922985
Cantidad de neuronas: 16 . Epochs: 10 . Learning rate: 0.001 . Accuracy: 0.9302 . Loss: 0.24212197411358358
Cantidad de neuronas: 16 . Epochs: 10 . Learning rate: 0.01 . Accuracy: 0.9498 . Loss: 0.16726445697247982
Cantidad de neuronas: 16 . Epochs: 10 . Learning rate: 0.1 . Accuracy: 0.9491 . Loss: 0.19335635280907154
Cantidad de neuronas: 32 . Epochs: 5 . Learning rate: 0.001 . Accuracy: 0.9221 . Loss: 0.2749239620089531
Cantidad de neuronas: 32 . Epochs: 5 . Learning rate: 0.01 . Accuracy: 0.9621 . Loss: 0.12019431031756103
Cantidad de neuronas: 32 . Epochs: 5 . Learning rate: 0.1 . Accuracy: 0.9559 . Loss: 0.16815929940771313
Cantidad de neuronas: 32 . Epochs: 10 . Learn

In [14]:
plot_accuracy_and_loss(hist_opt)

KeyError: 'accuracy'